<a href="https://colab.research.google.com/github/fabianobriao/Data_Scrubbing/blob/main/datascrubbing_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#https://bastter.com/mercado/Forum/865820/projeto-1-parte-3--python--limpeza-dos-dados-com-pandas